## Imports

In [ ]:
import argparse
import os
import sys
import warnings

import keras
import keras.preprocessing.image
import tensorflow as tf

## Set GPU Device

In [ ]:
from keras import backend as K

if 'tensorflow' == K.backend():
    import tensorflow as tf
from keras.backend.tensorflow_backend import set_session
GPUconfig = tf.ConfigProto()
GPUconfig.gpu_options.allow_growth = True
GPUconfig.gpu_options.visible_device_list = '1'
set_session(tf.Session(config=GPUconfig))

In [ ]:
from keras_retinanet import layers  # noqa: F401
from keras_retinanet import losses
from keras_retinanet import models
from keras_retinanet.callbacks import RedirectModel
from keras_retinanet.callbacks.eval import Evaluate
from keras_retinanet.models.retinanet import retinanet_bbox
from keras_retinanet.preprocessing.csv_generator import CSVGenerator
from keras_retinanet.preprocessing.kitti import KittiGenerator
from keras_retinanet.preprocessing.open_images import OpenImagesGenerator
from keras_retinanet.preprocessing.pascal_voc import PascalVocGenerator
from keras_retinanet.utils.anchors import make_shapes_callback
from keras_retinanet.utils.config import read_config_file, parse_anchor_parameters
from keras_retinanet.utils.keras_version import check_keras_version
from keras_retinanet.utils.model import freeze as freeze_model
from keras_retinanet.utils.transform import random_transform_generator
from keras_retinanet.utils.anchors import AnchorParameters

## Define Utility Functions

In [ ]:

def makedirs(path):
    # Intended behavior: try to create the directory,
    # pass if the directory exists already, fails otherwise.
    # Meant for Python 2.7/3.n compatibility.
    try:
        os.makedirs(path)
    except OSError:
        if not os.path.isdir(path):
            raise


def get_session():
    """ Construct a modified tf session.
    """
    config = tf.ConfigProto()
    config.gpu_options.allow_growth = True
    return tf.Session(config=config)


def model_with_weights(model, weights, skip_mismatch):
    """ Load weights for model.

    Args
        model         : The model to load weights for.
        weights       : The weights to load.
        skip_mismatch : If True, skips layers whose shape of weights doesn't match with the model.
    """
    if weights is not None:
        model.load_weights(weights, by_name=True, skip_mismatch=skip_mismatch)
    return model


def create_models(backbone_retinanet, num_classes, weights, multi_gpu=0,
                  freeze_backbone=False, lr=1e-4, config=None):
    """ Creates three models (model, training_model, prediction_model).

    Args
        backbone_retinanet : A function to call to create a retinanet model with a given backbone.
        num_classes        : The number of classes to train.
        weights            : The weights to load into the model.
        multi_gpu          : The number of GPUs to use for training.
        freeze_backbone    : If True, disables learning for the backbone.
        config             : Config parameters, None indicates the default configuration.

    Returns
        model            : The base model. This is also the model that is saved in snapshots.
        training_model   : The training model. If multi_gpu=0, this is identical to model.
        prediction_model : The model wrapped with utility functions to perform object detection (applies regression values and performs NMS).
    """

    modifier = freeze_model if freeze_backbone else None

    # load anchor parameters, or pass None (so that defaults will be used)
    anchor_params = None
    num_anchors   = None
    #if config and 'anchor_parameters' in config:
    if config:
        ratios,scales,sizes,strides = config
        anchor_params = AnchorParameters(sizes, strides, ratios, scales)
        num_anchors   = anchor_params.num_anchors()

    # Keras recommends initialising a multi-gpu model on the CPU to ease weight sharing, and to prevent OOM errors.
    # optionally wrap in a parallel model
    if multi_gpu > 1:
        from keras.utils import multi_gpu_model
        with tf.device('/cpu:1'):
            model = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = multi_gpu_model(model, gpus=multi_gpu)
    else:
        model          = model_with_weights(backbone_retinanet(num_classes, num_anchors=num_anchors, modifier=modifier), weights=weights, skip_mismatch=True)
        training_model = model

    # make prediction model
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)

    # compile model
    training_model.compile(
        loss={
            'regression'    : losses.smooth_l1(),
            'classification': losses.focal()
        },
        optimizer=keras.optimizers.adam(lr=lr, clipnorm=0.001)
    )

    return model, training_model, prediction_model


def create_callbacks(model, training_model, prediction_model, validation_generator, tensorboard_dir, evaluation, dataset_type, weighted_average, snapshots, snapshot_path, backbone_name):
    """ Creates the callbacks to use during training.

    Args
        model: The base model.
        training_model: The model that is used for training.
        prediction_model: The model that should be used for validation.
        validation_generator: The generator for creating validation data.
        args: parseargs args object.

    Returns:
        A list of callbacks used for training.
    """
    callbacks = []

    tensorboard_callback = None

    if tensorboard_dir:
        tensorboard_callback = keras.callbacks.TensorBoard(
            log_dir                = tensorboard_dir,
            histogram_freq         = 0,
            batch_size             = batch_size,
            write_graph            = True,
            write_grads            = False,
            write_images           = False,
            embeddings_freq        = 0,
            embeddings_layer_names = None,
            embeddings_metadata    = None
        )
        callbacks.append(tensorboard_callback)

    if evaluation and validation_generator:
        if dataset_type == 'coco':
            from ..callbacks.coco import CocoEval

            # use prediction model for evaluation
            evaluation = CocoEval(validation_generator, tensorboard=tensorboard_callback)
        else:
            evaluation = Evaluate(validation_generator, tensorboard=tensorboard_callback, weighted_average=weighted_average)
        evaluation = RedirectModel(evaluation, prediction_model)
        callbacks.append(evaluation)

    # save the model
    if snapshots:
        # ensure directory created first; otherwise h5py will error after epoch.
        makedirs(snapshot_path)
        checkpoint = keras.callbacks.ModelCheckpoint(
            os.path.join(
                snapshot_path,
                '{backbone}_{dataset_type}_{{epoch:02d}}.h5'.format(backbone=backbone_name, dataset_type=dataset_type)
            ),
            verbose=1,
            # save_best_only=True,
            # monitor="mAP",
            # mode='max'
        )
        checkpoint = RedirectModel(checkpoint, model)
        callbacks.append(checkpoint)

    callbacks.append(keras.callbacks.ReduceLROnPlateau(
        monitor    = 'loss',
        factor     = 0.1,
        patience   = 2,
        verbose    = 1,
        mode       = 'auto',
        min_delta  = 0.0001,
        cooldown   = 0,
        min_lr     = 0
    ))

    return callbacks


def create_generators(dataset_type, batch_size, config, image_min_side, image_max_side, preprocess_image):
    """ Create generators for training and validation.

    Args
        args             : parseargs object containing configuration for generators.
        preprocess_image : Function that preprocesses an image for the network.
    """
    common_args = {
        'batch_size'       : batch_size,
        'config'           : config,
        'image_min_side'   : image_min_side,
        'image_max_side'   : image_max_side,
        'preprocess_image' : preprocess_image,
    }

    # create random transform generator for augmenting training data
    if random_transform:
        transform_generator = random_transform_generator(
            min_rotation=-0.1,
            max_rotation=0.1,
            min_translation=(-0.1, -0.1),
            max_translation=(0.1, 0.1),
            min_shear=-0.1,
            max_shear=0.1,
            min_scaling=(0.9, 0.9),
            max_scaling=(1.1, 1.1),
            flip_x_chance=0.5,
            flip_y_chance=0.5,
        )
    else:
        transform_generator = random_transform_generator(flip_x_chance=0.5)

    if dataset_type == 'coco':
        # import here to prevent unnecessary dependency on cocoapi
        from ..preprocessing.coco import CocoGenerator

        train_generator = CocoGenerator(
            coco_path,
            'train2017',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = CocoGenerator(
            coco_path,
            'val2017',
            **common_args
        )
    elif dataset_type == 'pascal':
        train_generator = PascalVocGenerator(
            pascal_path,
            'trainval',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = PascalVocGenerator(
            pascal_path,
            'test',
            **common_args
        )
    elif dataset_type == 'csv':
        train_generator = CSVGenerator(
            annotations,
            classes,
            transform_generator=transform_generator,
            **common_args
        )

        if val_annotations:
            validation_generator = CSVGenerator(
                val_annotations,
                classes,
                **common_args
            )
        else:
            validation_generator = None
    elif dataset_type == 'oid':
        train_generator = OpenImagesGenerator(
            main_dir,
            subset='train',
            version=version,
            labels_filter=labels_filter,
            annotation_cache_dir=annotation_cache_dir,
            parent_label=parent_label,
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = OpenImagesGenerator(
            main_dir,
            subset='validation',
            version=version,
            labels_filter=labels_filter,
            annotation_cache_dir=annotation_cache_dir,
            parent_label=parent_label,
            **common_args
        )
    elif dataset_type == 'kitti':
        train_generator = KittiGenerator(
            kitti_path,
            subset='train',
            transform_generator=transform_generator,
            **common_args
        )

        validation_generator = KittiGenerator(
            kitti_path,
            subset='val',
            **common_args
        )
    else:
        raise ValueError('Invalid data type received: {}'.format(dataset_type))

    return train_generator, validation_generator

## Setup Parameters

In [ ]:
dataset_type = 'csv'
backbone_name = 'resnet50'
annotations = '' # path to train data csv file
classes = '' # path to class label csv file

batch_size = 8 #original = 4
epochs = 30 #original = 30
steps = 110 #original = 87
image_min_side = 800
image_max_side = 1333

config = None
random_transform = True

#Setup validation:
#evaluation = False
#val_annotations = None
#compute_val_loss = False
evaluation = True
val_annotations = '' # path to validation data csv file
compute_val_loss = True

snapshot = None
snapshots = False #disable saving snapshots
snapshot_path = './snapshots'
weights = None
imagenet_weights = True
no_weights = False
multi_gpu = 0
freeze_backbone = True
lr = 1e-4
tensorboard_dir = './logs'
weighted_average = True
workers = 1
max_queue_size = 10

## Setup Anchors (Optional)

In [ ]:
import numpy as np

In [ ]:
#Default Anchor Settings:
# sizes   = [32, 64, 128, 256, 512]
# strides = [8, 16, 32, 64, 128]
# ratios  = np.array([0.5, 1, 2], keras.backend.floatx())
# scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

In [ ]:
#Customized Anchor Settings:
sizes   = [32, 64, 128, 256, 512]
strides = [8, 16, 32, 64, 128]
ratios  = np.array([0.25, 1, 4], keras.backend.floatx())
scales  = np.array([2 ** 0, 2 ** (1.0 / 3.0), 2 ** (2.0 / 3.0)])

In [ ]:
config = [ratios,scales,sizes,strides] #uncomment to setup anchors

## Setup Model and Data

In [ ]:
# create object that stores backbone information
backbone = models.backbone(backbone_name)

# make sure keras is the minimum required version
check_keras_version()

# create the generators
train_generator, validation_generator = create_generators(dataset_type, batch_size, config, image_min_side, image_max_side, backbone.preprocess_image)

# create the model
if snapshot is not None:
    print('Loading model, this may take a second...')
    model            = models.load_model(snapshot, backbone_name=backbone_name)
    training_model   = model
    anchor_params    = None
    if config and 'anchor_parameters' in config:
        anchor_params = parse_anchor_parameters(config)
    prediction_model = retinanet_bbox(model=model, anchor_params=anchor_params)
else:
    weights = weights
    # default to imagenet if nothing else is specified
    if weights is None and imagenet_weights:
        weights = backbone.download_imagenet()

    print('Creating model, this may take a second...')
    model, training_model, prediction_model = create_models(
        backbone_retinanet=backbone.retinanet,
        num_classes=train_generator.num_classes(),
        weights=weights,
        multi_gpu=multi_gpu,
        freeze_backbone=freeze_backbone,
        lr=lr,
        config=config
    )
    print('Number of classes = {}'.format(train_generator.num_classes()))

# print model summary
#print(model.summary())

# this lets the generator compute backbone layer shapes using the actual backbone model
if 'vgg' in backbone_name or 'densenet' in backbone_name:
    train_generator.compute_shapes = make_shapes_callback(model)
    if validation_generator:
        validation_generator.compute_shapes = train_generator.compute_shapes

# create the callbacks
callbacks = create_callbacks(
    model,
    training_model,
    prediction_model,
    validation_generator,
    tensorboard_dir, 
    evaluation, 
    dataset_type, 
    weighted_average, 
    snapshot, 
    snapshot_path, 
    backbone
)

# Use multiprocessing if workers > 0
if workers > 0:
    use_multiprocessing = True
else:
    use_multiprocessing = False

if not compute_val_loss:
    validation_generator = None

In [ ]:
len(train_generator)

In [ ]:
len(validation_generator)

## Start Training

In [ ]:
keras.callbacks.EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto', baseline=None, restore_best_weights=False)

In [ ]:
# start training
training_model.fit_generator(
    generator=train_generator,
    steps_per_epoch=steps,
    epochs=epochs,
    verbose=1,
    callbacks=callbacks,
    workers=workers,
    use_multiprocessing=use_multiprocessing,
    max_queue_size=max_queue_size,
    validation_data=validation_generator
)

## Saving Model

In [ ]:
training_model.save('example.hdf5')

f = open('example.json', 'w')
model_json = prediction_model.to_json()
f.write(model_json)
f.close

## Change Model to Inference Model (source_dir, dest_dir)

In [ ]:
!keras_retinanet/bin/convert_model.py \
snapshots/example.hdf5 \
snapshots/example_inference.hdf5

## End